# 음악 review 내용 기반 추천 시스템 구현

In [46]:
import pandas as pd
from pandas import DataFrame as df
import numpy as np 
import os
import json 

# ratings = pd.read_csv('C:/Users/Affinity/total_data_3/total_data_3.csv')


# print(ratings.shape)

metadata=pd.read_csv('./IMDB-Movie-Data.csv')

indices=pd.Series(metadata.index, index=metadata['Title']).drop_duplicates()


In [54]:

df=ratings.iloc[:1000,1:]

df.head(10)



,amazon-id,songs_x,artist,root-genre,salesRank,reviewText,reviewerID,loudness,bpm,key,scale
0,630251228X,Jok Omako Nyako,Geoffrey Oryema,Alternative Rock,530408.0,"I give it 5 stars for the thumb piano, the gui...",A2D04UKNBWTSA3,0.863332,136.686081,E,major
1,630251228X,Jok Omako Nyako,Geoffrey Oryema,Alternative Rock,530408.0,This is my all-time favorite CD of African pop...,A3AWLFMKGRMI5M,0.863332,136.686081,E,major
2,630251228X,Jok Omako Nyako,Geoffrey Oryema,Alternative Rock,530408.0,His rhythms are impeccable. His voice is a smo...,A3PP84HBIN7NO7,0.863332,136.686081,E,major
3,630251228X,Jok Omako Nyako,Geoffrey Oryema,Alternative Rock,530408.0,His rhythms are impeccable. His voice is a sm...,A3PP84HBIN7NO7,0.863332,136.686081,E,major
4,630251228X,Jok Omako Nyako,Geoffrey Oryema,Alternative Rock,530408.0,"One does not often hear music from Uganda, and...",A3ET9X988R63C1,0.863332,136.686081,E,major
5,630251228X,Jok Omako Nyako,Geoffrey Oryema,Alternative Rock,530408.0,I have no idea what the words are but the musi...,A170A8NFQYJ90Z,0.863332,136.686081,E,major
6,630251228X,Jok Omako Nyako,Geoffrey Oryema,Alternative Rock,530408.0,Geoffrey Oryema is a relatively new artist for...,A1S9XVFBLBAXY5,0.863332,136.686081,E,major
7,630251228X,Jok Omako Nyako,Geoffrey Oryema,Alternative Rock,530408.0,Would recommend to all lovers of modern Africa...,A3VSDXC0WBNSIG,0.863332,136.686081,E,major
8,630251228X,Jok Omako Nyako,Geoffrey Oryema,Alternative Rock,530408.0,Oryema is one of a very small handful of artis...,A14CWMXH8UTAFF,0.863332,136.686081,E,major
9,630251228X,Jok Omako Nyako,Geoffrey Oryema,Alternative Rock,530408.0,This was one of the first Realworld releases I...,A26CK1EVKB866U,0.863332,136.686081,E,major


In [55]:
df['reviewText'][13]

"This is my all-time favorite CD of African popular music.  The tracks are alternately infectious and uptempo, and deeply moody and sorrowful -- apparently informed by Mr. Oryema's heartsickness over the political turmoil in his homeland.  The result is a consistently tasteful and affecting blend of cultural influences that has remained a top fave on my CD rack for many years.  (I actually played it for my boyfriend, a musician, on our first date, and we've been together almost ten years -- he found it to be a revelation as well.) I haven't felt quite the same connection with Mr. Oryema's later work, but &quot;Exile&quot; is a gem."

In [56]:
df['reviewText'].head(10)

0    I give it 5 stars for the thumb piano, the gui...
1    This is my all-time favorite CD of African pop...
2    His rhythms are impeccable. His voice is a smo...
3    His rhythms are impeccable.  His voice is a sm...
4    One does not often hear music from Uganda, and...
5    I have no idea what the words are but the musi...
6    Geoffrey Oryema is a relatively new artist for...
7    Would recommend to all lovers of modern Africa...
8    Oryema is one of a very small handful of artis...
9    This was one of the first Realworld releases I...
Name: reviewText, dtype: object

In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [58]:
tfidf=TfidfVectorizer(stop_words='english')

In [59]:
df.reviewText=df.reviewText.fillna('')

In [60]:
tfidf_matrix=tfidf.fit_transform(df.reviewText)
tfidf_matrix.shape

(1000, 2322)

In [61]:
from sklearn.metrics.pairwise import linear_kernel

In [62]:
cosine_sim=linear_kernel(tfidf_matrix, tfidf_matrix)

In [63]:
indices=pd.Series(df.index, index=df['songs_x']).drop_duplicates()
indices

songs_x
Jok Omako Nyako      0
Jok Omako Nyako      1
Jok Omako Nyako      2
Jok Omako Nyako      3
Jok Omako Nyako      4
Jok Omako Nyako      5
Jok Omako Nyako      6
Jok Omako Nyako      7
Jok Omako Nyako      8
Jok Omako Nyako      9
Jok Omako Nyako     10
Jok Omako Nyako     11
Makambo             12
Makambo             13
Makambo             14
Makambo             15
Makambo             16
Makambo             17
Makambo             18
Makambo             19
Makambo             20
Makambo             21
Makambo             22
Makambo             23
Piny Runa Woko      24
Piny Runa Woko      25
Piny Runa Woko      26
Piny Runa Woko      27
Piny Runa Woko      28
Piny Runa Woko      29
                  ... 
The One I Want     970
The One I Want     971
The One I Want     972
The One I Want     973
The One I Want     974
The One I Want     975
The One I Want     976
The One I Want     977
The One I Want     978
The One I Want     979
The One I Want     980
The One I Want     981
The

In [68]:
def rs(songs_x, n=10, sim=cosine_sim):
    idx=indices[songs_x]
    print(idx)
    sim_score=list(enumerate(cosine_sim[idx]))
    print(len(sim_score))
    print(sim_score[1])
    sim_score=sorted(sim_score, key=lambda x:x[1], reverse=True)
    sim_score=sim_score[1:(n+1)]
    

    movie_index=[i[0] for i in sim_score]
    return df.songs_x.iloc[movie_index]

In [69]:
rs('Piny Runa Woko')

Piny Runa Woko    24
Piny Runa Woko    25
Piny Runa Woko    26
Piny Runa Woko    27
Piny Runa Woko    28
Piny Runa Woko    29
Piny Runa Woko    30
Piny Runa Woko    31
Piny Runa Woko    32
Piny Runa Woko    33
Piny Runa Woko    34
Piny Runa Woko    35
dtype: int64
12
(1, array([0.09523316, 1.        , 0.13797551, 0.01594264, 0.04812272,
       0.01716463, 0.04643096, 0.03518307, 0.08035778, 0.08901363,
       0.07337453, 0.0477826 , 0.09523316, 1.        , 0.13797551,
       0.01594264, 0.04812272, 0.01716463, 0.04643096, 0.03518307,
       0.08035778, 0.08901363, 0.07337453, 0.0477826 , 0.09523316,
       1.        , 0.13797551, 0.01594264, 0.04812272, 0.01716463,
       0.04643096, 0.03518307, 0.08035778, 0.08901363, 0.07337453,
       0.0477826 , 0.09523316, 1.        , 0.13797551, 0.01594264,
       0.04812272, 0.01716463, 0.04643096, 0.03518307, 0.08035778,
       0.08901363, 0.07337453, 0.0477826 , 0.09523316, 1.        ,
       0.13797551, 0.01594264, 0.04812272, 0.01716463, 0.0

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()